# E-scale-train: This Notebook trains the NOIRE-Net E-region scaling networks

## 1 - Import libaries 

In [1]:
import os
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import pickle
from sklearn.model_selection import KFold

## 2 - Define function to get E-region scaling parameters from .par file (item1 = fE, item2 = hE)

In [2]:
# The function get_regression_label_from_par reads a .par file and returns 
# the E-region maximum frequency and the E-region height.
# Both the fE and hE must be values (can not be 'nan'). 

def get_regression_label_from_par(par_file_path):
    try:
        # Open the file at the specified path
        with open(par_file_path, 'r') as file:
            content = file.readline().strip()  # Read the first line and remove leading/trailing whitespace
            items = content.split()  # Split the line into individual items

            # Check if both the second and fourth items are not 'nan' (not a number)
            # If they are both valid numbers, convert them to floats and return them as a tuple
            if items[1].lower() != 'nan' and items[3].lower() != 'nan':
                return float(items[1]), float(items[3])
            else:
                # If either item is 'nan', return None
                return None
    except Exception as e:
        # Print an error message if an exception occurs while processing the file
        print(f"Error reading {par_file_path}: {e}")
        # Return None if there is an error
        return None


## 3 - Define function to load ionograms and preprocess the data

In [3]:
# The load_data function loads and preprocesses image data from a specified directory,
# converting images to grayscale and resizing them, while also extracting corresponding
# scaling parameters from associated .par files for a regression task.

def load_data(file_bases, ionogram_dir, parameter_dir):
    images = []  # List to store preprocessed images
    labels = []  # List to store corresponding regression labels
    
    # Iterate over the provided file bases to load images and labels
    for base in file_bases:
        img_path = os.path.join(ionogram_dir, base + '.png')
        par_path = os.path.join(parameter_dir, base + '.par')

        # Load the image, convert it to grayscale, resize it, and normalize pixel values
        image = load_img(img_path, color_mode='grayscale', target_size=(310, 310))
        image = img_to_array(image) / 255.0  # Normalize image pixels to be between 0 and 1

        # Get the regression labels from the .par file
        regression_label = get_regression_label_from_par(par_path)

        # Append the preprocessed image and regression label to their respective lists
        # Proceed only if valid regression labels are found
        if regression_label is not None:
            images.append(image)
            labels.append(regression_label)

    # Convert the lists of images and labels to numpy arrays and return them
    return np.array(images), np.array(labels)


## 4 - Define data paths and sort the files by their epoch timestamp

In [4]:
# This code defines paths to directories containing ionogram images and parameter files, 
# then creates a sorted list of unique identifiers (epoch timestamps) from the names of the ionogram images, 
# ensuring the data is ordered chronologically.

# Define the directory paths to the data
ionogram_dir = '/Users/akv020/Tensorflow/NOIRE-Net/data/train-val-test/ionograms'
parameter_dir = '/Users/akv020/Tensorflow/NOIRE-Net/data/train-val-test/parameters'

# Get all file names from the ionogram directory
all_files = [f.split('.')[0] for f in os.listdir(ionogram_dir) if f.endswith('.png')]

# Filter out files where the regression labels are not valid
data_files = []
for file_base in all_files:
    par_path = os.path.join(parameter_dir, file_base + '.par')
    if get_regression_label_from_par(par_path) is not None:
        data_files.append(file_base)

# Ensure the data is sorted by the epoch timestamp
data_files = sorted(data_files, key=int)

## 5 - Define training, validation and testing indeces for 10-fold cross-validation run

In [5]:
# This code sets up a 10-fold cross-validation framework, pre-computing and storing indices for 
# training, validation, and test sets for each fold, with a circular permutation applied 
# to the test set indices to maintain the temporal order of the data.

# Define the number of splits for KFold
n_splits = 10
kf = KFold(n_splits=n_splits)

# Pre-compute the indices for training, validation, and test sets for each fold
train_indices = []
val_indices = []
test_indices = []

# Number of samples for validation and test sets
num_val_samples = int(len(data_files) * 0.1)

for fold, (train_val_idx, test_idx) in enumerate(kf.split(data_files)):
    # Circular permutation for the test set
    test_idx = np.roll(test_idx, fold * num_val_samples)
    
    # Split the train_val_idx further into train and validation sets
    val_idx = train_val_idx[:num_val_samples]
    train_idx = train_val_idx[num_val_samples:]
    
    # Store the indices
    train_indices.append(train_idx)
    val_indices.append(val_idx)
    test_indices.append(test_idx)

## 6 - Define a function to create the NOIRE-Net architecture

In [6]:
# This code defines and complies NOIRE-Net a convolutional neural network (CNN) model using Keras, 
# with multiple convolutional layers, batch normalization, max pooling, and dense layers, 
# designed for binary classification tasks.

def NOIREnet():
    model = Sequential([
    # First convolutional layer with 32 filters and a kernel size of 3x3
    # 'padding=same' ensures the output size is the same as the input size
    # 'input_shape' is set for the first layer to indicate the shape of the input data
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(310, 310, 1)),
    
    # Batch normalization to normalize the activations from the previous layer
    BatchNormalization(),

    # Second convolutional layer with 32 filters and a kernel size of 3x3
    Conv2D(32, (3, 3), activation='relu'),

    # Another batch normalization
    BatchNormalization(),

    # First max pooling layer to reduce spatial dimensions
    MaxPooling2D((2, 2)),

    # Repeating the pattern of two convolutional layers followed by batch normalization
    # and a max pooling layer, gradually increasing the number of filters
    Conv2D(32, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Flatten the output from the convolutional layers to feed into dense layers
    Flatten(),

    # Dense (fully connected) layer with 256 neurons and relu activation
    Dense(256, activation='relu'),

    # Dropout layer to reduce overfitting
    Dropout(0.5),

    # Another dense layer with 128 neurons
    Dense(128, activation='relu'),

    # Output layer with a single neuron and sigmoid activation for binary classification
    Dense(2, activation='linear')
    ])
    
    # Compile the CNN model
    model.compile(
        optimizer='adam',  # Using the Adam optimizer for adaptive learning rate optimization
        loss='mse',  # Mean squared error loss function, suitable for regression tasks
        metrics=['mse']  # The model will report 'mse' as a performance metric
    )
    
    # Return the compiled model
    return model

## 6 - Train 10 CNNs for E-region scaling and save the models

In [7]:
# This code trains 10 Convolutional Neural Networks (CNNs) on differently split subsets
# of a dataset for binary classification,saves the best model of each training session, 
# and records their training histories.

# Create the directory for saving models and histories if it doesn't exist
save_dir = '/Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-scale'
os.makedirs(save_dir, exist_ok=True)

# Initialize lists to store the training histories and filenames of the best models
histories = []
model_filenames = []

# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,      # Factor to reduce the learning rate
    patience=15,     # Number of epochs with no improvement to wait before reducing LR
    min_lr=0.00001   # Minimum learning rate
)

# Loop to train 10 CNN models over 10 folds
for i in range(n_splits):
    # Load the training and validation sets for the current fold
    train_files = [data_files[idx] for idx in train_indices[i]]
    val_files = [data_files[idx] for idx in val_indices[i]]
    
    # Split the dataset into training, validation and testing sets
    X_train, y_train = load_data(train_files, ionogram_dir, parameter_dir)
    X_val, y_val = load_data(val_files, ionogram_dir, parameter_dir)
    
    
    # Create a new CNN model for each iteration
    model = NOIREnet()

    # Define the filename for the checkpoint model
    model_filename = os.path.join(save_dir, f'E-scale_run{i+1}.h5')

    # Define a checkpoint callback to save the best model based on validation accuracy
    checkpoint_callback = ModelCheckpoint(
        model_filename,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min',
        save_weights_only=False
    )

    # Train the model with specified callbacks including ReduceLROnPlateau
    history = model.fit(
        X_train, y_train,
        batch_size=64,
        epochs=60,
        validation_data=(X_val, y_val),
        shuffle=True,  # Enable shuffling
        callbacks=[checkpoint_callback, reduce_lr]  # Include ReduceLROnPlateau callback
    )

    # Save the training history and the filename of the saved best model
    histories.append(history.history)
    model_filenames.append(model_filename)
    
    print(f"Completed fold {i+1}/{n_splits}")

# Optionally, save the training histories to a file in the same 'E-scale' directory
history_filename = os.path.join(save_dir, 'training_histories.pkl')
with open(history_filename, 'wb') as file:
    pickle.dump({'histories': histories, 'model_filenames': model_filenames}, file)


Metal device set to: Apple M1 Max


2023-12-16 16:53:40.288955: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-16 16:53:40.289076: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/60


2023-12-16 16:53:41.176533: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-16 16:53:41.794882: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


137/137 [==============================] - ETA: 0s - loss: 711.5067 - mse: 711.5067

...


Epoch 1: val_loss improved from inf to 3458.28613, saving model to /Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-scale/E-scale_run9.h5
137/137 [==============================] - 46s 311ms/step - loss: 695.3378 - mse: 695.3378 - val_loss: 3458.2861 - val_mse: 3458.2861 - lr: 0.0010
Epoch 2/60
137/137 [==============================] - ETA: 0s - loss: 154.9029 - mse: 154.9029
Epoch 2: val_loss improved from 3458.28613 to 2453.72534, saving model to /Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-scale/E-scale_run9.h5
137/137 [==============================] - 44s 320ms/step - loss: 154.9029 - mse: 154.9029 - val_loss: 2453.7253 - val_mse: 2453.7253 - lr: 0.0010
Epoch 3/60
137/137 [==============================] - ETA: 0s - loss: 107.7523 - mse: 107.7523
Epoch 3: val_loss did not improve from 2453.72534
137/137 [==============================] - 45s 331ms/step - loss: 107.7523 - mse: 107.7523 - val_loss: 3166.0310 - val_mse: 3166.0310 - lr: 0.0010
Epoch 4/60
137/137 [===================

2023-12-17 05:02:18.115493: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


137/137 [==============================] - ETA: 0s - loss: 698.9858 - mse: 698.9858

2023-12-17 05:03:07.977022: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 4721.31787, saving model to /Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-scale/E-scale_run10.h5
137/137 [==============================] - 54s 374ms/step - loss: 698.9858 - mse: 698.9858 - val_loss: 4721.3179 - val_mse: 4721.3179 - lr: 0.0010
Epoch 2/60
137/137 [==============================] - ETA: 0s - loss: 145.0422 - mse: 145.0422
Epoch 2: val_loss improved from 4721.31787 to 3989.84424, saving model to /Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-scale/E-scale_run10.h5
137/137 [==============================] - 55s 404ms/step - loss: 145.0422 - mse: 145.0422 - val_loss: 3989.8442 - val_mse: 3989.8442 - lr: 0.0010
Epoch 3/60
137/137 [==============================] - ETA: 0s - loss: 125.5657 - mse: 125.5657
Epoch 3: val_loss improved from 3989.84424 to 3139.56787, saving model to /Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-scale/E-scale_run10.h5
137/137 [==============================] - 56s 407ms/step - loss: 125.5657 - mse: 125.5657